In [1]:
import numpy as np
import scipy.stats as stats
from utils import gibbs_sampling

In [2]:
# Model hyperparameters.
p = 2
K = 2

In [3]:
# Model parameters.
A1 = np.array([[0.29, 0.05], [0.32, 0.46]])
print('A1:')
print(A1)
A2 = np.array([[0.001, 0.15], [0.3, 0.34]])
print('A2:')
print(A2)
A = np.concatenate([A1, A2], axis=1)
print(A)
CovU = np.array([[9, 0], [0, 4]])
print('CovU')
print(CovU)

A1:
[[0.29 0.05]
 [0.32 0.46]]
A2:
[[0.001 0.15 ]
 [0.3   0.34 ]]
[[0.29  0.05  0.001 0.15 ]
 [0.32  0.46  0.3   0.34 ]]
CovU
[[9 0]
 [0 4]]


Chequeamos si cumple las condiciones requeridas:

In [4]:
#se chequea que la matriz CovU sea adecuada (semidefinida positiva)
eig_val_U = np.linalg.eigvals(CovU)
flag_sdp  = np.all(eig_val_U >= 0) and np.all(np.isreal(eig_val_U)) #valores propios no negativos y reales 
print(flag_sdp)

True


In [5]:
#se chequea que la matriz A sea adecuada (proceso estable, pag 15 Lutkepohl)
if(p==1): #no es necesario agregar bloque 
    A_test = A
else:
    A_block  = np.block(np.eye(K*(p-1)))
    A_zeros  = np.zeros((K*(p-1),K))
    A_bottom = np.concatenate((A_block,A_zeros),axis=1)
    A_test   = np.concatenate((A,A_bottom),axis=0)
eig_val_A   = np.absolute(np.linalg.eigvals(A_test))
print(eig_val_A)
flag_stable = np.all(eig_val_A < 1) #valores propios absolutos menores a 1
print(flag_stable)

[0.98621062 0.31920693 0.31920693 0.44443108]
True


In [6]:
# Real normalized data.
#[[0.87010704 0.89617185 0.94830146 1.10469029 1.1307551  1.18288471]
# [1.386914   1.54056019 1.61738329 1.64299099 1.66859869 1.66859869]]

In [7]:
# Initial values.
y0a = np.array([0.87010704, 1.386914])
y1a = np.array([0.89617185, 1.54056019])
y0b = np.array([0.94830146, 1.61738329])
y1b = np.array([1.10469029, 1.64299099])

In [8]:
def generate_data(A1, A2, covu, N, y0, y1):
    q = stats.multivariate_normal(mean=[0, 0], cov=covu)
    data = np.zeros((2, N))
    data[:, 0] = y0
    data[:, 1] = y1
    for i in range(2, N):
        new_y = np.dot(A1, y1) + np.dot(A2, y0) + q.rvs()
        data[:, i] = new_y
        y0 = y1
        y1 = new_y
    return data

In [9]:
# Fictional data.
data = generate_data(A1, A2, CovU, 1000, y0a, y1a)
#test_data_2 = generate_data(A1, A2, CovU, 1000, yb, yb)

In [10]:
# Prepare data.
X = np.zeros((K * p, data.shape[1] - p))
j = 0
for i in range(p, data.shape[1]):
    for t in range(p):
        X[t * K:(t + 1) * K, j] = data[:, (i - 1) - t]
    j += 1

Y0 = data[:, p:]

In [11]:
# Check data is right.
print(data.shape)
print(Y0.shape)
print(X.shape)

(2, 1000)
(2, 998)
(4, 998)


In [12]:
# Check data is right. x2
print(data[:, :6])
print(Y0[:, :6])
print(X[:, :6])

[[ 0.87010704  0.89617185 -0.67364212 -1.44787207 -2.17799927 -0.40400683]
 [ 1.386914    1.54056019  3.50662015  3.15947227 -3.44686048 -1.52371379]]
[[-0.67364212 -1.44787207 -2.17799927 -0.40400683 -1.30668912 -5.62527877]
 [ 3.50662015  3.15947227 -3.44686048 -1.52371379 -3.72624782 -2.67263122]]
[[ 0.89617185 -0.67364212 -1.44787207 -2.17799927 -0.40400683 -1.30668912]
 [ 1.54056019  3.50662015  3.15947227 -3.44686048 -1.52371379 -3.72624782]
 [ 0.87010704  0.89617185 -0.67364212 -1.44787207 -2.17799927 -0.40400683]
 [ 1.386914    1.54056019  3.50662015  3.15947227 -3.44686048 -1.52371379]]


In [13]:
# Run sampler.
DATA_PATH = '/home/chrisams/Documents/datasets/data_TAIM/processed/'
q = stats.norm
K = 2
p = 2
iters = 5
debug = False
mh_iters = 50
n_rows = 10000 # Number of rows of the data to load
method = 'normal'
init_mle = True

samples = gibbs_sampling(iters, DATA_PATH, K, p, q, mh_iters=mh_iters, init_mle=init_mle, n_rows=None,\
                         debug=False, method='normal', X=X, Y0=Y0)

Initializing parameters...
lk init: -860534932.5482124
Calculating MLE...
Init MLE theta calculated! (-17516.272218794773)
[[ 0.65725502  0.08728099 -0.09592984 -0.26315723]
 [-0.70250393  0.0469149   0.90798888  1.00653891]]
[[1.14682113 0.83307747]
 [0.83307747 5.78520451]]
Iteration 0
rejected: 47
accepted: 3
rejected: 46
accepted: 4
rejected: 49
accepted: 1
rejected: 43
accepted: 7
rejected: 48
accepted: 2
rejected: 46
accepted: 4
rejected: 48
accepted: 2
rejected: 50
accepted: 0
rejected: 45
accepted: 5
rejected: 49
accepted: 1
rejected: 48
accepted: 2
rejected: 45
accepted: 5
LK of new theta: -3940.029528137948
Time for iteration 0: 6.137133836746216
Iteration 1
rejected: 42
accepted: 8
rejected: 44
accepted: 6
rejected: 41
accepted: 9
rejected: 45
accepted: 5
rejected: 48
accepted: 2
rejected: 48
accepted: 2
rejected: 43
accepted: 7
rejected: 50
accepted: 0
rejected: 49
accepted: 1
rejected: 43
accepted: 7
rejected: 47
accepted: 3
rejected: 46
accepted: 4
LK of new theta: -3596.

In [14]:
# Print for comparison.
for sample in samples:
    print(sample)
    print('-' * 20)

[array([[ 0.82783098,  0.22895882, -0.10456639, -0.13366021],
       [-0.59934428, -0.09955308,  0.82332207,  1.00653891]]), array([[15.86108374,  0.09656991],
       [ 0.09656991, 20.77708368]])]
--------------------
[array([[ 0.82783098,  0.22895882, -0.10456639, -0.13366021],
       [-0.59934428, -0.09955308,  0.82332207,  1.00653891]]), array([[19.13062055, -7.87381988],
       [-7.87381988, 17.23641972]])]
--------------------
[array([[ 0.82783098,  0.22895882, -0.10456639, -0.13366021],
       [-0.59934428, -0.09955308,  0.82332207,  1.00653891]]), array([[12.95883929, -6.31019183],
       [-6.31019183, 16.30607269]])]
--------------------
[array([[ 0.82783098,  0.22895882, -0.10456639, -0.13366021],
       [-0.59934428, -0.09955308,  0.82332207,  1.00653891]]), array([[11.4602719 , -4.90970147],
       [-4.90970147, 13.13862295]])]
--------------------
[array([[ 0.82783098,  0.22895882, -0.10456639, -0.13366021],
       [-0.59934428, -0.09955308,  0.82332207,  1.00653891]]), arr

In [15]:
# Model parameters.
A1 = np.array([[0.29, 0.05], [0.32, 0.46]])
print('A1:')
print(A1)
A2 = np.array([[0.001, 0.15], [0.3, 0.34]])
print('A2:')
print(A2)
A = np.concatenate([A1, A2], axis=1)
print(A)
CovU = np.array([[9, 0], [0, 4]])
print('CovU')
print(CovU)

A1:
[[0.29 0.05]
 [0.32 0.46]]
A2:
[[0.001 0.15 ]
 [0.3   0.34 ]]
[[0.29  0.05  0.001 0.15 ]
 [0.32  0.46  0.3   0.34 ]]
CovU
[[9 0]
 [0 4]]
